# [실습] LangChain 기본 구조


LangChain을 활용하여 파이썬 프로그램 내에서 LLM을 활용해 보겠습니다.   

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 라이브러리 설치  
`langchain_openai`, `langchain_google_genai` 등의 라이브러리를 이용해 provider별 모델을 활용합니다.     
`langchain_huggingface`를 통해 오픈 모델을 연동할 수도 있습니다.

In [ ]:
!pip install langchain langchain_community openai langchain_openai rich

## API 키 설정   
OpenAI의 GPT 모델을 사용하기 위해, API 키를 환경 변수에 등록합니다.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

## LLM

LLM은 `ChatOpenAI`, `ChatGoogleGenerativeAI`와 같은 클래스로 불러올 수 있습니다.

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model = 'gpt-4.1-mini',

#                  # 최대 출력 토큰 수
#                  max_tokens = 4096,

#                  # 디코딩 파라미터(샘플링 확률 조절)

#                  temperature = 0.6,
#                  # 0~2 사이의 값, 낮을수록 높은 확률의 토큰 위주 출력
#                  top_p = 0.9,
#                  # 누적 확률 기준 0.9 내 토큰만 선택
#                  # top_k = 20
#                  # 확률 기준 상위 20개 토큰만 선택 (OpenAI에서는 미적용)

#                  )

llm  = ChatOpenAI(model='gpt-5-mini',
                  max_tokens=8192,
                  # Thinking 시간 설정('low','medium','high')
                  reasoning_effort='low'
                  )
# GPT-5는 디코딩 파라미터 설정이 제한적

In [ ]:
from rich import print as rprint
# 구조체는 rich를 통해 출력

rprint(llm)

## Prompt

LLM에 입력할 프롬프트는 다음의 방법으로 전달됩니다.

1. 단순 문자열
2. 랭체인 메시지 클래스
3. 프롬프트 템플릿과 입력 변수


### 1. 단순 문자열   

LLM은 `invoke()`를 통해 실행합니다.

In [ ]:
question = '''
# Instructions
최신 정보를 바탕으로 답변하세요.
현재 시장에 출시되었거나 학계에 발표된 LLM들 중에서, 기술적 완성도, 적용 범위의 광범위성, 그리고 혁신적인 기여도 측면에서 "가장 발전된"이라고 불릴 만한 모델들은 무엇입니까?
200자 정도의 결과물을 작성하세요.

# Output Format
다음 사항을 포함하여, 후보 모델들에 대해 짧게 개조식으로 작성하세요:
1.  해당 LLM의 이름과 출시 시기, 개발 주체
2.  주요 기술적 특징 (예: 아키텍처, 학습 방식, 파라미터 규모 등)
3.  다른 모델들과 비교했을 때 두드러지는 강점 또는 차별점
4.  어떤 근거(예: 공개된 벤치마크 결과, 논문, 실제 적용 사례 등)로 그렇게 판단하는지 설명
5.  해당 모델이 앞으로 어떤 영향을 미칠 것으로 예상되는지에 대한 간략한 전망
'''

response = llm.invoke(question)
response

출력 형식은 AIMessage 클래스입니다.   
반대로, 입력 문자열은 HumanMessage 클래스로 변환되어 전달됩니다.

In [ ]:
rprint(response)

메타데이터를 통해 토큰 사용량도 확인할 수 있습니다.

In [ ]:
print(response.content)

스트리밍을 통해 출력할 수도 있습니다.

In [ ]:
for chunk in llm.stream(question):
    print(chunk.content, end='')


batch를 통해 여러 개의 입력을 병렬적으로 처리할 수도 있습니다.

In [ ]:
topics = ['LLM이 무엇의 약자인가요? 20단어 이내로 답변하세요.',
          'LLM이랑 GPT랑 다른 건가요? 20단어 이내로 답변하세요.',
          'BERT와 GPT는 뭐가 다른가요? 20단어 이내로 답변하세요.']
results = llm.batch(topics)
results

reasoning 모델은 답변을 생성하기 전,   
생각(Thinking) 토큰을 먼저 생성하여 답변을 준비합니다.   
논리적 사고가 필요한 수학, 코딩 등의 문제에서 높은 성능을 보여줍니다.

In [ ]:
# 생각 과정 출력은 현재 제한된 공개
reasoning = {
    "effort": "low",  # 'low', 'medium', 'high'
    "summary": "detailed",  # 'detailed', 'auto', None
}

reasoning_llm = ChatOpenAI(model = 'gpt-5-mini',
                           max_tokens = 4096,

                           # reasoning_effort='low', # 'low', 'medium', 'high'
                           # 생각 과정 출력은 현재 제한된 공개
                           model_kwargs={"reasoning": reasoning}
                           )

In [ ]:
response = reasoning_llm.invoke(question)
rprint(response)

In [ ]:
print(response.content)

search-preview 모델은 검색 기능이 탑재된 모델입니다.   
해당 모델은 항상 웹 검색을 먼저 수행합니다.

In [ ]:
question = '''
# Instructions
2025년 10월 기준의 최신 정보를 바탕으로 답변하세요.
현재 시장에 출시되었거나 학계에 발표된 LLM들 중에서, 기술적 완성도, 적용 범위의 광범위성, 그리고 혁신적인 기여도 측면에서 "가장 발전된"이라고 불릴 만한 모델들은 무엇입니까?
400자 정도의 결과물을 작성하세요.
출처 링크를 명시하세요.

# Output Format
다음 사항을 포함하여, 후보 모델들에 대해 짧게 개조식으로 작성하세요:
1.  해당 LLM의 이름과 출시 시기, 개발 주체
2.  주요 기술적 특징 (예: 아키텍처, 학습 방식, 파라미터 규모 등)
3.  다른 모델들과 비교했을 때 두드러지는 강점 또는 차별점
4.  어떤 근거(예: 공개된 벤치마크 결과, 논문, 실제 적용 사례 등)로 그렇게 판단하는지 설명
5.  해당 모델이 앞으로 어떤 영향을 미칠 것으로 예상되는지에 대한 간략한 전망
'''

search_llm = ChatOpenAI(model = 'gpt-4o-search-preview', max_tokens = 4096)

response = search_llm.invoke(question)
response

In [ ]:
rprint(response)

In [ ]:
print(response.content)

Human Message 이외에도, 많은 LLM은 챗봇의 작동 방식을 결정하는 System Message를 지원합니다.

System Message는 보통 전체 대화의 첫 번째로 들어갑니다.

### 2. Message 클래스 전달하기   
클래스를 직접 생성하고 전달합니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    # SystemMessage('당신은 매우 논리적이고, 다양한 관점을 고려합니다.'),
    SystemMessage('당신은 답변을 세 문장으로만 합니다.'),
    HumanMessage('LLM이 인간을 대체할 수 있을까?')
]

response = llm.invoke(messages)
response

AIMessage를 함께 전달하는 방식으로, 멀티-턴 대화를 수행할 수 있습니다.

In [ ]:
messages.append(response)
messages.append(HumanMessage('완전히 대체하는 건 언제가 될까?'))
# System, Human, AI, Human

response2 = llm.invoke(messages)
response2

### 3. Prompt Template

프롬프트 템플릿을 사용하면, 정해진 템플릿에 입력 변수의 공간을 설정할 수 있습니다.

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [ ]:
explain_prompt = PromptTemplate(template = """LLM과 {subject}의 공통점을 5문장으로 설명하세요.""")
prompt = explain_prompt.format(subject = "트랜스포머 네트워크")
prompt

In [ ]:
response = llm.invoke(prompt)
print(response.content)

입력 변수가 여러 개일 수도 있습니다.

In [ ]:
explain_template = "{topic}에 대해 {style}로 설명하세요."
explain_prompt2 = PromptTemplate(template = explain_template)

prompt = explain_prompt2.format(topic='LLM의 미래', style='부정적인 전망으')
prompt

In [ ]:
llm.invoke(prompt).content

System, AI 등의 메시지를 포함하기 위해서는 ChatPromptTemplate를 사용합니다.

In [ ]:
chat_prompt = ChatPromptTemplate([
    ("system", '당신은 항상 이모지로만 대답합니다.'),
    ("user", '{topic}에 대해 설명해주세요.')
    # 역할은 4개 (user = human), (ai = assistant)
]
)
prompt = chat_prompt.format_messages(topic='랭체인')
prompt


In [ ]:
llm.invoke(prompt)

Few-Shot Prompt Template은 프롬프트에 예시를 추가할 수 있습니다.

In [ ]:
# 예시 : Prompt Example 2개

from langchain.prompts.few_shot import FewShotPromptTemplate

example_prompt = PromptTemplate(
    # input_variables=["question", "answer"],
    template="질문: {question}\n답변: {answer}"
)

examples = [
    {
        "question": "고객 불만을 어떻게 처리해야 하나요?",
        "answer": "1) 경청하기 2) 공감 표현 3) 해결책 제시 4) 후속 조치"
    },
    {
        "question": "회의를 효율적으로 진행하려면?",
        "answer": "1) 명확한 안건 설정 2) 시간 관리 3) 참여 유도 4) 결론 정리"
    }
]

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 예시를 참고해서 답변해주세요:",
    suffix="질문: {input}\n답변:",
)


print(few_shot_prompt.format(input="휴가 신청은 어떻게 해야 하나요?"))

In [ ]:
question = "휴가 신청은 어떻게 해야 하나요?"
X = few_shot_prompt.format(input=question)
print(llm.invoke(X).content)

Chat Message에 Few Shot을 적용하는 경우, 아래와 같이 나타납니다.

In [ ]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate(
        [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(

    example_prompt=example_prompt,
    examples=examples,
)

few_shot_prompt.format_messages()


In [ ]:
final_prompt = ChatPromptTemplate(
    [
        ("system", "다음 예시를 참고하여 같은 형식으로 답변하세요."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
final_prompt.format_messages(input="휴가 신청은 어떻게 해야 하나요?")


### 멀티모달 프롬프트 전달하기

멀티모달 모델은 이미지의 URL이나 실제 파일을 프롬프트에 전달할 수 있습니다.

In [ ]:
# 예제 이미지: 타임테이블

import base64
import httpx
from langchain_core.messages import HumanMessage

image_url = "https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/timetable.png"

response = httpx.get(image_url)
image_data = base64.b64encode(response.content).decode("utf-8")

with open('timetable.png', 'wb') as file:
    file.write(response.content)

이미지 URL을 전달합니다.

In [ ]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "이 그림에 대해 설명해주세요."},
        {
            "type": "image_url",
            "image_url": {"url": image_url},
        },
    ]
)

ai_msg = llm.invoke([message])
print(ai_msg.content)

오프라인 파일은 조금 복잡한 포맷으로 전달합니다.

In [ ]:
with open('./timetable.png', 'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# 멀티모달 모델의 프롬프팅은 조금 더 구체적이어야 합니다.

message = HumanMessage(
    content=[
        {"type": "text", "text": "이 그림에 적힌 장소와 시간을 모두 찾아서 표 형태로 출력하세요. 총 몇 개입니까?"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)

ai_msg = llm.invoke([message])
print(ai_msg.content)